In [42]:
from tkinter import *
from PIL import Image, ImageTk
import sys
import os
import cv2
import numpy as np
sys.path.append(r'C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages')
from cvzone.HandTrackingModule import HandDetector

class GUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Hand Gesture Controlled presentations")
        self.root.geometry("1280x720")
        self.root.config(background="powder blue")
        self.root.state('zoomed')  # Fullscreen mode

        self.main_frame = Frame(self.root, bg="teal")
        self.main_frame.pack(expand=True, fill=BOTH)

        self.center_frame = Frame(self.main_frame, bg="teal", bd=10, relief=GROOVE)
        self.center_frame.place(relx=0.5, rely=0.5, anchor=CENTER)

        self.ABC2 = LabelFrame(self.center_frame, bd=12, font=("Courier", 20, 'bold', "roman"), text="Slides", bg="teal", relief=GROOVE)
        self.ABC2.grid(row=1, column=0, pady=5, padx=5, sticky=N)

        ABC1 = Frame(self.center_frame, bd=20, padx=100, bg="teal", relief=GROOVE)
        ABC1.grid(row=0, column=0, columnspan=4, sticky=W+E)  # Stretch across the width
        self.lblTitle = Label(ABC1, text="Hand Gesture Controlled Presentation", font=("Courier", 20, 'bold', "roman"), pady=9, bd=5, width=40, bg="teal", fg="white")
        self.lblTitle.pack(expand=True)  # Center the title text

        # Canvas for OpenCV image
        self.canvas = Canvas(self.ABC2, width=900, height=500)
        self.canvas.grid(row=0, column=0)

        self.update_frame()

    def update_frame(self):
        # Camera Setup
        cap = cv2.VideoCapture(0)
        cap.set(3, 1280)
        cap.set(4, 720)

        # Hand Detector
        detectorHand = HandDetector(detectionCon=0.1, maxHands=1)

        # Variables
        imgNumber = 0
        buttonPressed = False
        counter = 0
        annotations = [[]]
        annotationNumber = -1
        annotationStart = False

        # Get list of presentation images
        folderPath = r"C:\Users\DELL\Desktop\Slides"
        pathImages = sorted(os.listdir(folderPath), key=len)

        def show_frame():
            nonlocal imgNumber, buttonPressed, counter, annotations, annotationNumber, annotationStart

            # Get image frame
            success, img = cap.read()
            img = cv2.flip(img, 1)
            pathFullImage = os.path.join(folderPath, pathImages[imgNumber])
            imgCurrent = cv2.imread(pathFullImage)

            # Find the hand and its landmarks
            hands, img = detectorHand.findHands(img)
            # Draw Gesture Threshold line
            cv2.line(img, (0, 300), (1280, 300), (0, 255, 0), 10)

            if hands and not buttonPressed:
                hand = hands[0]
                cx, cy = hand["center"]
                lmList = hand["lmList"]
                fingers = detectorHand.fingersUp(hand)

                # Constrain values for easier drawing
                xVal = int(np.interp(lmList[8][0], [1280 // 2, 1280], [0, 1280]))
                yVal = int(np.interp(lmList[8][1], [150, 720-150], [0, 720]))
                indexFinger = xVal, yVal

                if cy <= 300:
                    if fingers == [1, 0, 0, 0, 0]:
                        buttonPressed = True
                        print("Left")
                        if imgNumber > 0:
                            imgNumber -= 1
                            annotations = [[]]
                            annotationNumber = -1
                            annotationStart = False
                    if fingers == [0, 0, 0, 0, 1]:
                        print("Right")
                        buttonPressed = True
                        if imgNumber < len(pathImages) - 1:
                            imgNumber += 1
                            annotations = [[]]
                            annotationNumber = -1
                            annotationStart = False

                if fingers == [0, 1, 1, 0, 0]:
                    cv2.circle(imgCurrent, indexFinger, 12, (0, 0, 255), cv2.FILLED)

                if fingers == [0, 1, 0, 0, 0]:
                    if not annotationStart:
                        annotationStart = True
                        annotationNumber += 1
                        annotations.append([])
                    print(annotationNumber)
                    annotations[annotationNumber].append(indexFinger)
                    cv2.circle(imgCurrent, indexFinger, 12, (0, 0, 255), cv2.FILLED)
                else:
                    annotationStart = False

                if fingers == [0, 1, 1, 1, 0]:
                    if annotations:
                        annotations.pop(-1)
                        annotationNumber -= 1
                        buttonPressed = True

            else:
                annotationStart = False

            if buttonPressed:
                counter += 1
                if counter > 30:
                    counter = 0
                    buttonPressed = False

            for annotation in annotations:
                for j in range(1, len(annotation)):
                    cv2.line(imgCurrent, annotation[j - 1], annotation[j], (0, 0, 200), 12)

            small_width, small_height = 900, 500  # Adjust these values as needed

            # Resize imgCurrent for smaller presentation size
            imgCurrent = cv2.resize(imgCurrent, (small_width, small_height))
            
            # Resize imgSmall for smaller size
            imgSmall = cv2.resize(img, (int(213), int(120)))
            
            h, w, _ = imgCurrent.shape
            imgCurrent[0:120, w - 213: w] = imgSmall

            imgCurrent = cv2.cvtColor(imgCurrent, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(imgCurrent)
            imgtk = ImageTk.PhotoImage(image=img_pil)
            self.canvas.create_image(0, 0, anchor=NW, image=imgtk)
            self.canvas.imgtk = imgtk
            self.root.after(10, show_frame)

        show_frame()
        key = cv2.waitKey(1)
        if key == ord('q'):
            self.root.quit()
            

if __name__ == '__main__':
    root = Tk()
    application = GUI(root)
    root.mainloop()



In [46]:
from tkinter import *
from tkinter import messagebox  # Import messagebox
from PIL import Image, ImageTk
import sys
import os
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

class GUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Hand Gesture Controlled Presentations")
        self.root.geometry("1280x720")
        self.root.config(background="powder blue")
        self.root.state('zoomed')  # Fullscreen mode

        self.main_frame = Frame(self.root, bg="teal")
        self.main_frame.pack(expand=True, fill=BOTH)

        self.center_frame = Frame(self.main_frame, bg="teal", bd=10, relief=GROOVE)
        self.center_frame.place(relx=0.5, rely=0.5, anchor=CENTER)

        self.ABC2 = LabelFrame(self.center_frame, bd=12, font=("Courier", 20, 'bold', "roman"), text="Slides", bg="teal", relief=GROOVE, width=1100)
        self.ABC2.grid(row=1, column=0, pady=5, padx=5, sticky=N)

        ABC1 = Frame(self.center_frame, bd=20, padx=100, bg="teal", relief=GROOVE)
        ABC1.grid(row=0, column=0, columnspan=4, sticky=W + E)
        
        # Adjusted pady to a single integer for less gap
        self.lblTitle = Label(ABC1, text="Hand Gesture Controlled Presentation", font=("Courier", 20, 'bold', "roman"), bd=5, width=40, bg="teal", fg="Cornsilk")
        self.lblTitle.pack(expand=True)
        
        # Add small text label below the title
        self.author_label = Label(ABC1, text="By Malik Zeshan", font=("Courier", 12, 'italic'), bg="teal", fg="Cornsilk")
        self.author_label.pack() # This will now be closer to the title


        # Canvas for OpenCV image
        self.canvas = Canvas(self.ABC2, width=1000, height=500)
        self.canvas.grid(row=0, column=0)

        # Label to instruct user to add the presentation path
        self.instruction_label = Label(self.canvas, text="➕ Add the presentation path below", font=("Courier", 18, 'bold'), fg="Black")
        self.instruction_label.place(relx=0.5, rely=0.5, anchor=CENTER)  # Center the label in the canvas

        # Entry box for file location
        self.entry_frame = Frame(self.main_frame, bg="teal", bd=10, relief=GROOVE)
        self.entry_frame.pack(side=BOTTOM, fill=X)
        self.entry_label = Label(self.entry_frame, text="Slides Location:", font=("Courier", 14, "bold"), bg="teal", fg="Cornsilk")
        self.entry_label.pack(side=LEFT, padx=(100, 0))

        self.file_location_entry = Entry(self.entry_frame, font=("Courier", 12), width=50)
        self.file_location_entry.pack(side=LEFT, padx=10, pady=10)

        # Button to start the presentation
        self.start_button = Button(self.entry_frame, text="Start Presentation", command=self.start_presentation, font=("Courier", 12, "bold"), bg="teal", fg="white")
        self.start_button.pack(side=LEFT, padx=10)

        # Center the entry frame content
        self.entry_frame.pack_configure(padx=20)
        self.entry_label.pack(anchor=CENTER)
        self.file_location_entry.pack(anchor=CENTER)

    def start_presentation(self):
        folderPath = self.file_location_entry.get().strip()
        
        # Validate the folder path
        if not os.path.isdir(folderPath):
            messagebox.showerror("Error", "Invalid folder path. Please enter a valid path.")  # Show error message
            return

        pathImages = sorted(os.listdir(folderPath), key=len)
        if not pathImages:
            messagebox.showerror("Error", "No images found in the specified folder.")  # Show error message
            return

        # Hide the instruction label when the presentation starts
        self.instruction_label.place_forget()

        self.update_frame(folderPath, pathImages)

    def update_frame(self, folderPath, pathImages):
        # Camera Setup
        cap = cv2.VideoCapture(0)
        cap.set(3, 1280)
        cap.set(4, 720)

        # Hand Detector
        detectorHand = HandDetector(detectionCon=0.1, maxHands=1)

        # Variables
        imgNumber = 0
        buttonPressed = False
        counter = 0
        annotations = [[]]
        annotationNumber = -1
        annotationStart = False

        def show_frame():
            nonlocal imgNumber, buttonPressed, counter, annotations, annotationNumber, annotationStart

            # Get image frame
            success, img = cap.read()
            img = cv2.flip(img, 1)
            pathFullImage = os.path.join(folderPath, pathImages[imgNumber])
            imgCurrent = cv2.imread(pathFullImage)

            # Find the hand and its landmarks
            hands, img = detectorHand.findHands(img)
            # Draw Gesture Threshold line
            cv2.line(img, (0, 300), (1280, 300), (0, 255, 0), 10)

            if hands and not buttonPressed:
                hand = hands[0]
                cx, cy = hand["center"]
                lmList = hand["lmList"]
                fingers = detectorHand.fingersUp(hand)

                # Constrain values for easier drawing
                xVal = int(np.interp(lmList[8][0], [1280 // 2, 1280], [0, 1280]))
                yVal = int(np.interp(lmList[8][1], [150, 720-150], [0, 720]))
                indexFinger = xVal, yVal

                if cy <= 300:
                    if fingers == [1, 0, 0, 0, 0]:
                        buttonPressed = True
                        print("Left")
                        if imgNumber > 0:
                            imgNumber -= 1
                            annotations = [[]]
                            annotationNumber = -1
                            annotationStart = False
                    if fingers == [0, 0, 0, 0, 1]:
                        print("Right")
                        buttonPressed = True
                        if imgNumber < len(pathImages) - 1:
                            imgNumber += 1
                            annotations = [[]]
                            annotationNumber = -1
                            annotationStart = False

                if fingers == [0, 1, 1, 0, 0]:
                    cv2.circle(imgCurrent, indexFinger, 12, (0, 0, 255), cv2.FILLED)

                if fingers == [0, 1, 0, 0, 0]:
                    if not annotationStart:
                        annotationStart = True
                        annotationNumber += 1
                        annotations.append([])
                    print(annotationNumber)
                    annotations[annotationNumber].append(indexFinger)
                    cv2.circle(imgCurrent, indexFinger, 12, (0, 0, 255), cv2.FILLED)
                else:
                    annotationStart = False

                if fingers == [0, 1, 1, 1, 0]:
                    if annotations:
                        annotations.pop(-1)
                        annotationNumber -= 1
                        buttonPressed = True

            else:
                annotationStart = False

            if buttonPressed:
                counter += 1
                if counter > 30:
                    counter = 0
                    buttonPressed = False

            for annotation in annotations:
                for j in range(1, len(annotation)):
                    cv2.line(imgCurrent, annotation[j - 1], annotation[j], (0, 0, 200), 12)

            small_width, small_height = 1000, 500  # Adjust these values as needed

            # Resize imgCurrent for smaller presentation size
            imgCurrent = cv2.resize(imgCurrent, (small_width, small_height))
            
            # Resize imgSmall for smaller size
            imgSmall = cv2.resize(img, (int(213), int(120)))
            
            h, w, _ = imgCurrent.shape
            imgCurrent[0:120, w - 213: w] = imgSmall

            imgCurrent = cv2.cvtColor(imgCurrent, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(imgCurrent)
            imgtk = ImageTk.PhotoImage(image=img_pil)
            self.canvas.create_image(0, 0, anchor=NW, image=imgtk)
            self.canvas.imgtk = imgtk
            self.root.after(10, show_frame)

        show_frame()
        key = cv2.waitKey(1)
        if key == ord('q'):
            self.root.quit()

if __name__ == '__main__':
    root = Tk()
    application = GUI(root)
    root.mainloop()
